In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [0]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score
import time

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/sets_de_datos/dima_train_with_features_6.csv",
                    dtype={ \
                            'antiguedad' : np.float32, 
                            'habitaciones' : np.float32, 
                            'garages' : np.float32, 
                            'banos' : np.float32,
                            'metroscubiertos' : np.float32, 
                            'metrostotales' : np.float32, 
                            'idzona' : np.float32, 
                            'lat' : np.float32, 
                            'lng' : np.float32, 
                            'gimnasio' : np.float32, 
                            'usosmultiples' : np.float32, 
                            'piscina' : np.float32, 
                            'escuelascercanas' : np.float32,
                            'centroscomercialescercanos' : np.float32, 
                            'precio' : np.float32, 
                            'servicios_cercanos' : np.float32,
                            'utilidades_extra' : np.float32, 
                            'tipodepropiedad_mean_antiguedad' : np.float32,
                            'tipodepropiedad_mean_habitaciones ' : np.float32, 
                            'tipodepropiedad_mean_garages' : np.float32,
                            'tipodepropiedad_mean_banos' : np.float32, 
                            'tipodepropiedad_mean_metroscubiertos' : np.float32,
                            'tipodepropiedad_mean_utilidades_extra' : np.float32,
                            'tipodepropiedad_mean_servicios_cercanos' : np.float32, 
                            'ciudad_mean_antiguedad' : np.float32,
                            'ciudad_mean_habitaciones' : np.float32, 
                            'ciudad_mean_garages' : np.float32, 
                            'ciudad_mean_banos': np.float32,
                            'ciudad_mean_metroscubiertos' : np.float32, 
                            'ciudad_mean_utilidades_extra' : np.float32,
                            'ciudad_mean_servicios_cercanos' : np.float32, 
                            'provincia_mean_antiguedad' : np.float32,
                            'provincia_mean_habitaciones' : np.float32, 
                            'provincia_mean_garages' : np.float32,
                            'provincia_mean_banos' : np.float32, 
                            'provincia_mean_metroscubiertos' : np.float32,
                            'provincia_mean_utilidades_extra' : np.float32, 
                            'provincia_mean_servicios_cercanos' : np.float32,
                            'ciudad_mean_lat' : np.float32, 
                            'ciudad_mean_lng' : np.float32, 
                            'provincia_mean_lat' : np.float32,
                            'provincia_mean_lng' : np.float32, 
                            'coseno_distancia_angular_al_centro_ciudad' : np.float32,
                            'coseno_distancia_angular_al_centro_provincia' : np.float32,
                            'coseno_distancia_angular_al_centro_pais' : np.float32, 
                            'año' : np.float32, 
                            'mes' : np.float32, 
                            'dia' : np.float32,
                            'metros_no_cubiertos' : np.float32, 
                            'cantidad_espacios' : np.float32, 
                            'metros_x_espacio' : np.float32,
                            'metros_x_habitaciones' : np.float32, 
                            'metros_x_garages' : np.float32, 
                            'metros_x_banos' : np.float32,
                            'metros_x_utilidades_extra' : np.float32, 
                            'ratio_metros_cubiertos' : np.float32,
                            'distancia_minima_Terreno_comercial' : np.float32,
                            'distancia_minima_Local_Comercial' : np.float32,
                            'distancia_minima_Oficina_comercial' : np.float32,
                            'distancia_minima_Local_en_centro_comercial' : np.float32,
                            'distancia_minima_Bodega_comercial' : np.float32, 
                            'banos_preciopromedio_ciudad' : np.float32,
                            'habitaciones_preciopromedio_ciudad' : np.float32, 
                            'garages_preciopromedio_ciudad' : np.float32,
                            'banos_preciopromedio_metroscubiertos' : np.float32,
                            'habitaciones_preciopromedio_metroscubiertos' : np.float32,
                            'garages_preciopromedio_metroscubiertos' : np.float32, 
                            'precio_x_m2' : np.float32,
                            'tipodepropiedad_mean_precio' : np.float32, 
                            'titulo_cantidad_palabras_importantes' : np.float32,
                            'descripcion_cantidad_palabras_importantes' : np.float32,
                            'direccion_cantidad_palabras_importantes' : np.float32,
                            'titulo_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'descripcion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'direccion_cantidad_caracteres_en_palabras_importantes' : np.float32,
                            'titulo_longitud_media_de_palabra' : np.float32,
                            'descripcion_longitud_media_de_palabra' : np.float32,
                            'direccion_longitud_media_de_palabra' : np.float32, 
                            'titulo_cantidad_stopwords' : np.float32,
                            'descripcion_cantidad_stopwords' : np.float32, 
                            'direccion_cantidad_stopwords' : np.float32,
                            'titulo_cantidad_signos_puntacion' : np.float32,
                            'descripcion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_signos_puntacion' : np.float32,
                            'direccion_cantidad_palabras_en_mayuscula' : np.float32,
                            'direccion_cantidad_titulos' : np.float32, 
                            'titulo_cantidad_palabras_top_k' : np.float32,
                            'descripcion_cantidad_palabras_top_k' : np.float32,
                            'direccion_cantidad_palabras_top_k' : np.float32,
                            'titulo_cantidad_palabras_bottom_k' : np.float32,
                            'descripcion_cantidad_palabras_bottom_k' : np.float32,
                            'direccion_cantidad_palabras_bottom_k' : np.float32,
                            'titulo_cantidad_prefijos_top_k' : np.float32, 
                            'descripcion_cantidad_prefijos_top_k' : np.float32,
                            'direccion_cantidad_prefijos_top_k' : np.float32, 
                            'titulo_cantidad_postfijos_top_k' : np.float32,
                            'descripcion_cantidad_postfijos_top_k' : np.float32,
                            'direccion_cantidad_postfijos_top_k' : np.float32, 
                            '0' : np.float32, 
                            '1' : np.float32, 
                            '2' : np.float32, 
                            '3' : np.float32, 
                            '4' : np.float32, 
                            '5' : np.float32, 
                            '6' : np.float32,
                            '7' : np.float32,
                            'distancia_euclideana_al_origen' : np.float32, 
                            'distancia_minima_comercial' : np.float32,
                            'coseno_maximo_ciudad_pais' : np.float32,
                            'ciudad_mean_antiguedad_sobre_provincia_mean_antiguedad' : np.float32,
                            'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra' : np.float32,
                            'antiguedad_sobre_tipodepropiedad_mean_antiguedad' : np.float32,
                            'direccion_cantidad_al_menos_una_mayuscula' : np.float32,
                            'direccion_cantidad_fijos_top_k' : np.float32, 
                            'titulo_cantidad_fijos_top_k' : np.float32,
                            'titulo_palabras_top_k_sobre_total_palabras' : np.float32,
                            'ciudad_distancia_al_origen':  np.float32, 
                            'ciudad_mean_mean_todas' : np.float32,
                            'ciudad_mean_antiguedad_sobre_mean_metrocubiertos' : np.float32
                            }
                    )

In [0]:
train.set_index('id', inplace = True)

In [0]:
TARGET = 'precio'
FEATURES = ['antiguedad', 'tipodepropiedad_mean_garages', 'metros_x_espacio',
       'metros_x_habitaciones',
       'antiguedad_sobre_tipodepropiedad_mean_antiguedad',
       'distancia_minima_Local_en_centro_comercial', 'lng',
       'coseno_distancia_angular_al_centro_ciudad',
       'tipodepropiedad_mean_metroscubiertos',
       'distancia_minima_Bodega_comercial',
       'distancia_minima_Oficina_comercial', 'ciudad_mean_habitaciones',
       'metros_x_banos', 'lat',
       'tipodepropiead_mean_utilidades_extra_sobre_ciudad_mean_utilidades_extra',
       'metros_x_garages', 'tipodepropiedad_mean_antiguedad', 'idzona', 'año',
       'habitaciones_preciopromedio_ciudad', 'metroscubiertos', 'precio_x_m2',
       'metrostotales', 'garages_preciopromedio_metroscubiertos',
       'habitaciones_preciopromedio_metroscubiertos',
       'garages_preciopromedio_ciudad', 'banos_preciopromedio_metroscubiertos',
       'banos_preciopromedio_ciudad']

In [0]:
X = train[FEATURES]
y = train[TARGET]

In [0]:
K = 5
def objective(hyper_parametros):

    rf_regressor = RandomForestRegressor(**hyper_parametros)
    
    score = cross_val_score(rf_regressor, X, y, 
                            scoring='neg_mean_absolute_error', cv=K).mean()
    
    print("SCORE: {:.3f} params {}".format(score, hyper_parametros))
    
    return {'loss': -score, 'status': STATUS_OK}

In [0]:
space = {
    'max_depth': hp.randint("max_depth", 4) + 16,  
    'max_features': hp.randint("max_features", 3) + 12, 
    'n_estimators': hp.randint("n_estimators", 300) + 200,
    'min_samples_leaf': hp.randint('min_samples_leaf', 1000) + 1,
    'min_samples_split' : hp.randint('min_samples_split', 1000) + 2,
}

In [24]:
N_ITER = 20
t0 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=N_ITER)
t1 = time.time()
print('Tiempo = {0:.2f} horas'.format((t1 - t0)/3600))

SCORE: -683946.869 params {'max_depth': 18, 'max_features': 13, 'min_samples_leaf': 269, 'min_samples_split': 174, 'n_estimators': 292}
SCORE: -694960.802 params {'max_depth': 19, 'max_features': 14, 'min_samples_leaf': 156, 'min_samples_split': 918, 'n_estimators': 338}
SCORE: -702080.436 params {'max_depth': 17, 'max_features': 12, 'min_samples_leaf': 429, 'min_samples_split': 424, 'n_estimators': 478}
SCORE: -687685.132 params {'max_depth': 19, 'max_features': 14, 'min_samples_leaf': 300, 'min_samples_split': 79, 'n_estimators': 220}
SCORE: -736163.731 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 949, 'min_samples_split': 454, 'n_estimators': 358}
SCORE: -679564.510 params {'max_depth': 18, 'max_features': 14, 'min_samples_leaf': 154, 'min_samples_split': 581, 'n_estimators': 266}
SCORE: -695417.183 params {'max_depth': 18, 'max_features': 14, 'min_samples_leaf': 257, 'min_samples_split': 875, 'n_estimators': 393}
SCORE: -688480.199 params {'max_depth': 17, 'max_

SCORE: -683946.869 params {'max_depth': 18, 'max_features': 13, 'min_samples_leaf': 269, 'min_samples_split': 174, 'n_estimators': 292}  
SCORE: -694960.802 params {'max_depth': 19, 'max_features': 14, 'min_samples_leaf': 156, 'min_samples_split': 918, 'n_estimators': 338}  
SCORE: -702080.436 params {'max_depth': 17, 'max_features': 12, 'min_samples_leaf': 429, 'min_samples_split': 424, 'n_estimators': 478}  
SCORE: -687685.132 params {'max_depth': 19, 'max_features': 14, 'min_samples_leaf': 300, 'min_samples_split': 79, 'n_estimators': 220}  
SCORE: -736163.731 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 949, 'min_samples_split': 454, 'n_estimators': 358}  
SCORE: -679564.510 params {'max_depth': 18, 'max_features': 14, 'min_samples_leaf': 154, 'min_samples_split': 581, 'n_estimators': 266}  
SCORE: -695417.183 params {'max_depth': 18, 'max_features': 14, 'min_samples_leaf': 257, 'min_samples_split': 875, 'n_estimators': 393}  
SCORE: -688480.199 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 304, 'min_samples_split': 372, 'n_estimators': 269}  
SCORE: -713469.316 params {'max_depth': 16, 'max_features': 12, 'min_samples_leaf': 567, 'min_samples_split': 660, 'n_estimators': 486}  
SCORE: -698932.781 params {'max_depth': 16, 'max_features': 14, 'min_samples_leaf': 404, 'min_samples_split': 499, 'n_estimators': 252}  
SCORE: -727773.913 params {'max_depth': 16, 'max_features': 13, 'min_samples_leaf': 791, 'min_samples_split': 730, 'n_estimators': 205}  
SCORE: -715130.048 params {'max_depth': 16, 'max_features': 13, 'min_samples_leaf': 598, 'min_samples_split': 625, 'n_estimators': 326}  
SCORE: -735886.658 params {'max_depth': 19, 'max_features': 14, 'min_samples_leaf': 956, 'min_samples_split': 927, 'n_estimators': 470}  
SCORE: -724691.271 params {'max_depth': 19, 'max_features': 12, 'min_samples_leaf': 738, 'min_samples_split': 68, 'n_estimators': 400}  
SCORE: -729346.475 params {'max_depth': 19, 'max_features': 14, 'min_samples_leaf': 830, 'min_samples_split': 528, 'n_estimators': 451}  
SCORE: -699953.134 params {'max_depth': 19, 'max_features': 13, 'min_samples_leaf': 410, 'min_samples_split': 391, 'n_estimators': 359}  
SCORE: -733359.961 params {'max_depth': 19, 'max_features': 13, 'min_samples_leaf': 895, 'min_samples_split': 237, 'n_estimators': 485}  
SCORE: -678951.071 params {'max_depth': 18, 'max_features': 13, 'min_samples_leaf': 232, 'min_samples_split': 141, 'n_estimators': 315}  
SCORE: -736399.129 params {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 960, 'min_samples_split': 842, 'n_estimators': 421}  
SCORE: -732190.235 params {'max_depth': 19, 'max_features': 12, 'min_samples_leaf': 871, 'min_samples_split': 546, 'n_estimators': 301}  

In [0]:
#params = {'max_depth': 17, 'max_features': 13, 'min_samples_leaf': 8, 'min_samples_split': 5, 'n_estimators': 456}